# Clustering Coffee Shops in Toronto
##### To help stakeholder decide location of new shop

In [6]:
import requests
import pandas as pd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [81]:
CLIENT_ID='WEY5QTBDI52OQPPIT1LFTUODUP2C2VK0RC3EOJNA3RGVR1CS'
CLIENT_SECRET='EFTHBJDVGW0P0C3WKMYSKLP4UQNLWNDMFBGEGYXZ40ED0EMS'
VERSION='20180605'
geo=Nominatim(user_agent="my_app")
location=geo.geocode('Toronto')
lat=location.latitude
lon=location.longitude



In [3]:
import sys
!{sys.executable} -m pip install geopy

## Calling the Foursquare API with necessary details included

In [11]:
url='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}&query={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    lon,
    50,
    'coffee'    
)
url

'https://api.foursquare.com/v2/venues/search?&client_id=WEY5QTBDI52OQPPIT1LFTUODUP2C2VK0RC3EOJNA3RGVR1CS&client_secret=EFTHBJDVGW0P0C3WKMYSKLP4UQNLWNDMFBGEGYXZ40ED0EMS&v=20180605&ll=43.653963,-79.387207&limit=50&query=coffee'

## The following steps are implemented in the code:
1. Get JSON response from API call
2. Parse the JSON to extract the names, latitude and longitude of coffee shops.
3. Create a Pandas dataframe out of the information


In [67]:
result=requests.get(url).json()
venues=result['response']['venues']
coffee_lat=[]
coffee_lon=[]
coffee_name=[]
coffee_neigh=[]
for i in range(len(venues)):
    coffee_lat.append(venues[i]['location']['lat'])
    coffee_lon.append(venues[i]['location']['lng'])
    coffee_name.append(venues[i]['name'])
data={'name':coffee_name,'lat':coffee_lat,'lon':coffee_lon}
df_coffee=pd.DataFrame(data)
df_coffee.head()

,name,lat,lon
0,HotBlack Coffee,43.650364,-79.388669
1,Balzac's Coffee,43.657854,-79.379200
2,Jimmy's Coffee,43.654493,-79.401311
3,Timothy's World Coffee,43.656219,-79.383290
4,Timothy's World Coffee,43.654270,-79.387448


In [69]:
import sys
!{sys.executable} -m pip install folium

     |████████████████████████████████| 92kB 281kB/s eta 0:00:01


In [70]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

### The locations of coffee shops all around Toronto are clustered into 5 different groups based on their latitude and longitude co-ordinates. 
This will help the stakeholder understand the segmentation of coffee shops around Toronto to give them a better idea of where to open a new coffee shop. They may choose to open a shop in the areas where coffee shop clusters are less dense for less competition.
KNN clustering algorithm is used.

In [80]:
map_lat=df_coffee['lat'][0]
map_lon=df_coffee['lon'][0]
toronto_map=folium.Map(location=[map_lat,map_lon],zoom_start=13)
km=KMeans(n_clusters=5, random_state=4).fit(df_coffee[['lat','lon']])
x=np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
df_coffee.insert(0,"cluster",km.labels_)

for lat,long,name,cluster in zip(df_coffee['lat'],df_coffee['lon'],df_coffee['name'],df_coffee['cluster']):
    label='{}'.format(name)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat,long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)        
    

toronto_map

(The visual map may not display in github. In that case open it in watson studio with the link : https://eu-gb.dataplatform.cloud.ibm.com/analytics/notebooks/v2/fc474ab9-66b0-4c58-9f5c-8d732c5fc4bc/view?access_token=83fd6133064e311906951cf75356f1ecc38088ebc6628e0da190ab339b3c3ef9)
A screenshot of the clustered map is also provided in the github repo under the name 'clustered map'